In [6]:
lung_cancer = {
#      'v3_trimmed': 's3://olgabot-maca/lung_cancer/sourmash_v3/',
    
    # Only v2 has many (4k) signatures
               'v2': 's3://olgabot-maca/lung_cancer/sourmash_v2/',
#               'v1': 's3://olgabot-maca/lung_cancer/sourmash_v2/'
              }

tabula_muris = {
#     'v2_untrimmed': 's3://olgabot-maca/facs/sourmash_v2/',
#                 'v3_trimmed': 's3://olgabot-maca/facs/sourmash_v3/',
                
                # Only v1 has signatures
                 'v1': 's3://olgabot-maca/facs/sourmash/'}

In [12]:
data = dict(lung_cancer=lung_cancer, tabula_muris=tabula_muris)
sourmash_runs = pd.DataFrame(data)
sourmash_runs = sourmash_runs.unstack().reset_index()
sourmash_runs = sourmash_runs.rename(columns={'level_0':'dataset', 'level_1':'run_version', 0: 's3_path'})
sourmash_runs = sourmash_runs.dropna()
sourmash_runs.head()

,dataset,run_version,s3_path
1,lung_cancer,v2,s3://olgabot-maca/lung_cancer/sourmash_v2/
2,tabula_muris,v1,s3://olgabot-maca/facs/sourmash/


In [13]:
import pandas as pd

data = {"trimmed": dict(v1=False, v2=False, v3=True), 
        'ksizes': dict(v1='21,31,52', v2='21,33,51', v3='21,33,51'),
        
        # Was a protein signature calculated?
        'protein': dict(v1=False, v2=True, v3=True),
        
        # Was a DNA signature calculated?
         'dna': dict(v1=True, v2=True, v3=True)}

metadata = pd.DataFrame(data)
metadata.head()

,trimmed,ksizes,protein,dna
v1,False,"21,31,52",False,True
v2,False,"21,33,51",True,True
v3,True,"21,33,51",True,True


Example command:

```
reflow run ~/reflow-workflows/sourmash_compare.rf -signatures=s3://olgabot-maca/facs/sourmash/ -output==s3://olgabot-maca/facs/sourmash_index_k21 -ksize=21
```

## Sourmash compare

In [40]:

import json

lines = []

program = 'compare'
folder = f'/home/ubuntu/kmer-hashing/sourmash/{program}'
! mkdir -p $folder


for i, row in sourmash_runs.iterrows():
    run_id = f'{row.dataset}_{row.run_version}'.format(**row)

    ksizes = metadata.loc[row.run_version].ksizes.split(',')

    for ksize in ksizes:
        output = row.s3_path.rstrip('/') + f'_{program}_k{ksize}'
        line = {'id': f'{run_id}_k{ksize}', 'signatures': row.s3_path, 'output': output, 'ksize': ksize}
        lines.append(line)
            
samples = pd.DataFrame.from_records(lines)
samples.to_csv(f'{folder}/samples.csv', index=False)
samples

config = 	{
		"program": f"/home/ubuntu/reflow-workflows/sourmash_{program}.rf",
		"runs_file": "samples.csv"
	}



with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! ls -lha $folder

total 21M
drwxrwxr-x 2 ubuntu ubuntu 4.0K Jun 18 22:00 .
drwxrwxr-x 8 ubuntu ubuntu 4.0K Jun 18 22:01 ..
-rw-rw-r-- 1 ubuntu ubuntu   92 Jun 18 22:01 config.json
-rw------- 1 ubuntu ubuntu   72 Jun 18 21:23 log.0
-rw------- 1 ubuntu ubuntu   72 Jun 18 21:23 log.1
-rw------- 1 ubuntu ubuntu   72 Jun 18 21:23 log.2
-rw------- 1 ubuntu ubuntu   72 Jun 18 21:23 log.3
-rw------- 1 ubuntu ubuntu   72 Jun 18 21:23 log.4
-rw------- 1 ubuntu ubuntu   72 Jun 18 21:23 log.5
-rw------- 1 ubuntu ubuntu 1.6M Jun 18 21:28 log.lung_cancer_v2
-rw------- 1 ubuntu ubuntu   89 Jun 18 22:01 log.lung_cancer_v2_k21
-rw------- 1 ubuntu ubuntu   89 Jun 18 22:01 log.lung_cancer_v2_k33
-rw------- 1 ubuntu ubuntu   89 Jun 18 22:01 log.lung_cancer_v2_k51
-rw------- 1 ubuntu ubuntu  19M Jun 18 21:30 log.tabula_muris_v1
-rw------- 1 ubuntu ubuntu   90 Jun 18 22:01 log.tabula_muris_v1_k21
-rw------- 1 ubuntu ubuntu   90 Jun 18 22:01 log.tabula_muris_v1_k31
-rw------- 1 ubuntu ubuntu   90 Jun 18 22:01 log.tabula_muris

## Sourmash index

In [39]:
lines = []

program = 'index'
folder = f'/home/ubuntu/kmer-hashing/sourmash/{program}'
! mkdir -p $folder
print(folder)


for i, row in sourmash_runs.iterrows():
    run_id = f'{row.dataset}_{row.run_version}'.format(**row)

    ksizes = metadata.loc[row.run_version].ksizes.split(',')

    for ksize in ksizes:
        output = row.s3_path.rstrip('/') + f'_{program}_k{ksize}'
        line = {'id': f'{run_id}_k{ksize}', 'signatures': row.s3_path, 'output': output, 'ksize': ksize}
        lines.append(line)
            
samples = pd.DataFrame.from_records(lines)
samples.to_csv(f'{folder}/samples.csv', index=False)

config = 	{
		"program": f"/home/ubuntu/reflow-workflows/sourmash_{program}.rf",
		"runs_file": "samples.csv"
	}



with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)

samples

! ls -lha $folder

/home/ubuntu/kmer-hashing/sourmash/index
total 16K
drwxrwxr-x 2 ubuntu ubuntu 4.0K Jun 18 22:01 .
drwxrwxr-x 8 ubuntu ubuntu 4.0K Jun 18 22:01 ..
-rw-rw-r-- 1 ubuntu ubuntu   90 Jun 18 22:01 config.json
-rw-rw-r-- 1 ubuntu ubuntu  672 Jun 18 22:01 samples.csv


In [38]:
folder

'/home/ubuntu/kmer-hashing/sourmash/compare'